In [2]:
from bs4 import BeautifulSoup as bs
import regex

#lista nazw plików html

listaplikow = []

for i in range(1,10):
    listaplikow.append(f"flights0{i}.html")

for i in range(10,35):
    listaplikow.append(f"flights{i}.html")
listaplikow

['flights01.html',
 'flights02.html',
 'flights03.html',
 'flights04.html',
 'flights05.html',
 'flights06.html',
 'flights07.html',
 'flights08.html',
 'flights09.html',
 'flights10.html',
 'flights11.html',
 'flights12.html',
 'flights13.html',
 'flights14.html',
 'flights15.html',
 'flights16.html',
 'flights17.html',
 'flights18.html',
 'flights19.html',
 'flights20.html',
 'flights21.html',
 'flights22.html',
 'flights23.html',
 'flights24.html',
 'flights25.html',
 'flights26.html',
 'flights27.html',
 'flights28.html',
 'flights29.html',
 'flights30.html',
 'flights31.html',
 'flights32.html',
 'flights33.html',
 'flights34.html']

In [3]:
numerlotu = []
skad = []
dokad = []

for plik in listaplikow:
    #otwieranie pliku
    HTMLFile = open(plik, "r", encoding="utf-8")
    # read
    index = HTMLFile.read() 
    # tworzenie parsera
    Parse = bs(index, 'html')
    # znajdowanie hrefów z linkami zawierającymi informacje o lotach
    lista2=[]
    for a in Parse.find_all('a', href=regex.compile("^\/flights\/lo[a-zA-Z0-9]*")):
        lista2.append(str(a['href']))
    # splitting i tworzenie oddzielnych list z kazdego lotu na podstawie linków
    for i in lista2:
        lista = (i.split("/"))
        lista.pop()
        #usuwa ostatni i pierwszy element listy (były puste)
        lista.pop(0)
        lista.pop(0)
        
        numerlotu.append(lista[0])
        skad.append(lista[1])
        dokad.append(lista[2])


In [4]:
import pandas as pd

In [5]:
df = pd.DataFrame(list(zip(numerlotu, skad, dokad)), columns=['numer_lotu', 'skad', 'dokad'])
df

,numer_lotu,skad,dokad
0,lo6250,antalya,warsaw
1,lo724,tbilisi,warsaw
2,lo728,yerevan,warsaw
3,lo3910,krak%c3%b3w,warsaw
4,lo5222,vienna,warsaw
...,...,...,...
7548,lo3969,warsaw,lublin
7549,lo3803,warsaw,rzesz%c3%b3w
7550,lo10,chicago,krak%c3%b3w
7551,lo159,krak%c3%b3w,tel_aviv


In [6]:
#dokąd najwięcej
df.groupby("dokad").agg(
    ile = ("dokad", "count")
).sort_values(by="ile", axis='rows', ascending=False)

,ile
dokad,
warsaw,4085
krak%c3%b3w,229
gda%c5%84sk,180
wroc%c5%82aw,147
vilnius,126
...,...
strasbourg,1
puerto_plata,1
reykjav%c3%adk,1


In [7]:
#skad najwiecej
df.groupby("skad").agg(
    ile = ("skad", "count")
).sort_values(by="ile", axis='rows', ascending=False)

,ile
skad,
warsaw,3137
krak%c3%b3w,282
gda%c5%84sk,200
wroc%c5%82aw,192
vilnius,156
...,...
goose_bay,1
reykjav%c3%adk,1
strasbourg,1


In [8]:
df["skad"] = df["skad"].apply(
    lambda x: x.capitalize()
)

In [9]:
df["dokad"] = df["dokad"].apply(
    lambda x: x.capitalize()
)

In [10]:
df["skad"] = df["skad"].apply(
    lambda x: x.capitalize()
)

In [11]:
def konwersja_underscore_na_spacje(wartosc_ze_spacjami):
    underscore="_"
    nowa_wartosc=wartosc_ze_spacjami.replace(underscore, " ")
    return nowa_wartosc

In [12]:
def konwersja_polskich_znakow(wartosc_bez_znakow):
    nowa_wartosc = wartosc_bez_znakow.replace("%c5%82", "ł")
    nowa_wartosc = nowa_wartosc.replace("%c3%b3", "ó")
    nowa_wartosc = nowa_wartosc.replace("%c5%84", "ń")
    nowa_wartosc = nowa_wartosc.replace("%c3%bc", "ü")
    nowa_wartosc = nowa_wartosc.replace("%c3%ad", "i")
    nowa_wartosc = nowa_wartosc.replace("%c5%a1", "s")
    nowa_wartosc = nowa_wartosc.replace("%c5%9f", "s")
    nowa_wartosc = nowa_wartosc.replace("%c4%83", "a")
    nowa_wartosc = nowa_wartosc.replace("%c3%ba", "ú")
    nowa_wartosc = nowa_wartosc.replace("%c3%b6", "ö")
    nowa_wartosc = nowa_wartosc.replace("%c4%b0", "I")
    nowa_wartosc = nowa_wartosc.replace("%c3%a1", "a")
    return nowa_wartosc

In [13]:
df["skad"] = df["skad"].apply(konwersja_underscore_na_spacje)

In [14]:
df["dokad"] = df["dokad"].apply(konwersja_underscore_na_spacje)

In [15]:
df["skad"] = df["skad"].apply(konwersja_polskich_znakow)

In [16]:
df["dokad"] = df["dokad"].apply(konwersja_polskich_znakow)

In [17]:
df.replace("Goleniow", "Szczecin", inplace=True)
#lotnisko w Szczecinie to Szczecin-Goleniów
df.replace("Poland", "Olsztyn", inplace=True)
#nie czytał Olsztyna jako lotniska (Szymany), tylko wartość "Poland"

In [18]:
df
df["lot_miasta"] = df[['skad', 'dokad']].agg('-'.join, axis=1)

In [19]:
df

,numer_lotu,skad,dokad,lot_miasta
0,lo6250,Antalya,Warsaw,Antalya-Warsaw
1,lo724,Tbilisi,Warsaw,Tbilisi-Warsaw
2,lo728,Yerevan,Warsaw,Yerevan-Warsaw
3,lo3910,Kraków,Warsaw,Kraków-Warsaw
4,lo5222,Vienna,Warsaw,Vienna-Warsaw
...,...,...,...,...
7548,lo3969,Warsaw,Lublin,Warsaw-Lublin
7549,lo3803,Warsaw,Rzeszów,Warsaw-Rzeszów
7550,lo10,Chicago,Kraków,Chicago-Kraków
7551,lo159,Kraków,Tel aviv,Kraków-Tel aviv


In [20]:
df.to_csv("tabelazlotami-airportia.csv", index=False)

In [21]:
tabelakoncowa = df.groupby("lot_miasta").agg(
    liczbalotow = ("lot_miasta", "count"),
).sort_values(by="liczbalotow", axis="rows", ascending=False)


In [22]:
tabelakoncowa

,liczbalotow
lot_miasta,
Kraków-Warsaw,219
Wrocław-Warsaw,191
Warsaw-Kraków,174
Gdańsk-Warsaw,170
Warsaw-Wrocław,144
...,...
Strasbourg-Warsaw,1
Catania-Warsaw,1
Budapest-Newark,1


In [23]:
lacznaliczbalotow = tabelakoncowa["liczbalotow"].sum()

In [24]:
tabelakoncowa["procentlotow"] = (tabelakoncowa['liczbalotow'] / lacznaliczbalotow ) * 100

In [25]:
tabelakoncowa.head(17)

,liczbalotow,procentlotow
lot_miasta,,
Kraków-Warsaw,219,2.899510
Wrocław-Warsaw,191,2.528797
Warsaw-Kraków,174,2.303720
Gdańsk-Warsaw,170,2.250761
Warsaw-Wrocław,144,1.906527
Warsaw-Gdańsk,142,1.880048
Rzeszów-Warsaw,141,1.866808
Vilnius-Warsaw,133,1.760890
Budapest-Warsaw,124,1.641732


In [26]:
#jaki procent stanowią loty krajowe, a jaki międzynarodowe?
df['skad'].unique()

array(['Antalya', 'Tbilisi', 'Yerevan', 'Kraków', 'Vienna', 'Tel aviv',
       'Istanbul', 'Amsterdam', 'Vilnius', 'Athens', 'Wrocław', 'Newark',
       'Budapest', 'Prague', 'Copenhagen', 'Munich', 'Lisbon', 'Miami',
       'Mumbai', 'Frankfurt-am-main', 'Stockholm', 'Düsseldorf', 'Zurich',
       'New york', 'Geneva', 'Toronto', 'Milan', 'Oslo', 'Katowice',
       'Rzeszów', 'Paris', 'Colombo', 'Gdańsk', 'Chicago', 'London',
       'Riga', 'Heraklion', 'Tokyo', 'Cluj-napoca', 'Ljubljana',
       'Tallinn', 'Podgorica', 'Bucharest', 'Chisinau', 'Sofia', 'Tirana',
       'Belgrad', 'Bydgoszcz', 'Szczecin', 'Poznań', 'Kosice',
       'Barcelona', 'Madrid', 'Zagreb', 'Berlin', 'Venezia', 'Hamburg',
       'Dubrovnik', 'Brussels', 'Billund', 'Luxembourg', 'Nice',
       'Palma de mallorca', 'Ostrava', 'Lublin', 'Gothenburg', 'Babimost',
       'New delhi', 'Los angeles', 'Faro', 'Warsaw', 'Seoul', 'Paphos',
       'Bodrum', 'Asunción', 'Strasbourg', 'Puerto plata', 'Cancún',
       'Rhode